In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv
import json

from library_dicom.dicom_processor.model.csv_reader.CsvReader import CsvReader

In [ ]:
#AHL nifti PT + csv 

In [2]:
liste_inference = os.listdir('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference')

In [3]:
liste = []
for inference in liste_inference : 
    subliste = []
    subliste.append('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference'+'/'+inference)
    subliste.append(inference.split('_')[0])
    liste.append(subliste)


In [4]:
liste[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/2.16.840.1.113669.632.20.210791.20000053223_mask_pred_float.nii',
 '2.16.840.1.113669.632.20.210791.20000053223']

In [5]:
csv_ahl = '/media/deeplearning/Elements/AHL2011_NIFTI.csv'

with open(csv_ahl, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [6]:
csv_data[0]

['CON,DA',
 '13011101331015',
 '1.2.124.113532.10.149.172.5.20131107.93455.7725981',
 'pet0',
 '/media/storage/AHL2011_NIFTI/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_CT.nii',
 '/media/storage/AHL2011_NIFTI/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_PT.nii',
 '/media/storage/AHL2011_NIFTI/1.2.124.113532.10.149.172.5.20131107.93455.7725981_nifti_mask.nii']

In [7]:
#garder que les pet 0
pet0 = []
for row in csv_data :
    if 'pet0' in row : 
        pet0.append(row)

#find pet 
for study in liste : 
    for pet in pet0 : 
        if study[1] in pet[5] :
            study.append(pet[1]) #patient id
            study.append(pet[5]) #pet path 
            

In [8]:
len(liste)

404

In [9]:
liste[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/2.16.840.1.113669.632.20.210791.20000053223_mask_pred_float.nii',
 '2.16.840.1.113669.632.20.210791.20000053223',
 '13011101221017',
 '/media/storage/AHL2011_NIFTI/2.16.840.1.113669.632.20.210791.20000053223_nifti_PT.nii']

In [10]:
#change pet path 

for serie in liste: 
    serie[3] = serie[3].replace('storage', 'deeplearning')

In [11]:
liste[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/2.16.840.1.113669.632.20.210791.20000053223_mask_pred_float.nii',
 '2.16.840.1.113669.632.20.210791.20000053223',
 '13011101221017',
 '/media/deeplearning/AHL2011_NIFTI/2.16.840.1.113669.632.20.210791.20000053223_nifti_PT.nii']

In [12]:
for serie in liste : 
    path_pet = serie[-1].split('/')
    new_path = '/media/deeplearning/Elements'+'/'+path_pet[3]+'/'+path_pet[4]

    serie[3] = new_path

In [13]:
liste[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/2.16.840.1.113669.632.20.210791.20000053223_mask_pred_float.nii',
 '2.16.840.1.113669.632.20.210791.20000053223',
 '13011101221017',
 '/media/deeplearning/Elements/AHL2011_NIFTI/2.16.840.1.113669.632.20.210791.20000053223_nifti_PT.nii']

#find associate csv 
csv_csv = '/media/deeplearning/Elements/AHL2011_CSV_DATA/pet0.csv'

with open(csv_csv, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data_2 = []
    for row in reader :
        csv_data_2.append(row)
        
del csv_data_2[0] #enlever première ligne


csv_data_2[0]

In [14]:
#clean dataset 
json_path = '/media/deeplearning/Elements/AHL2011_NIFTI/json/AHL2011_final_list_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

589


In [17]:
csv_directory = '/media/deeplearning/Elements/AHL2011_CSV_DATA/AHL2011-CSV'

for serie in liste : 
    for data in dataset : 
        if serie[1] in data[0] :
            serie.append(os.path.join(csv_directory, data[-1]))

In [18]:
liste[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/2.16.840.1.113669.632.20.210791.20000053223_mask_pred_float.nii',
 '2.16.840.1.113669.632.20.210791.20000053223',
 '13011101221017',
 '/media/deeplearning/Elements/AHL2011_NIFTI/2.16.840.1.113669.632.20.210791.20000053223_nifti_PT.nii',
 '/media/deeplearning/Elements/AHL2011_CSV_DATA/AHL2011-CSV/poi_is_jun 5_2013-EV.csv']

In [20]:
#difference volume 
from library_dicom.dicom_processor.tools.csv_volume import *
for serie in liste : 
    serie.append(get_difference_volume(serie[-1]))

In [21]:
liste[3]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/1.2.840.113704.1.111.4268.1351517946.9_mask_pred_float.nii',
 '1.2.840.113704.1.111.4268.1351517946.9',
 '13011101021014',
 '/media/deeplearning/Elements/AHL2011_NIFTI/1.2.840.113704.1.111.4268.1351517946.9_nifti_PT.nii',
 '/media/deeplearning/Elements/AHL2011_CSV_DATA/AHL2011-CSV/zou_ab_oct 29_2012_SK.csv',
 20.72999999999999]

In [22]:
#write csv 
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/post_processing'

with open(os.path.join(nifti_directory, 'ahl_post_process.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID", "NIFTI_PET", "NIFTI_MASK", "CSV", "DIFF"])
    for serie in liste: 
        csv_writer.writerow([serie[2], serie[1], serie[3], serie[0], serie[4], serie[5]])

In [ ]:
#plus gros écart 

In [25]:
csv_post = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/post_processing/ahl_post_process.csv'

with open(csv_post, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data_3 = []
    for row in reader :
        csv_data_3.append(row)
        
del csv_data_3[0] #enlever première ligne


In [26]:
csv_data_3[0]

['13011101221017',
 '2.16.840.1.113669.632.20.210791.20000053223',
 '/media/deeplearning/Elements/AHL2011_NIFTI/2.16.840.1.113669.632.20.210791.20000053223_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/2.16.840.1.113669.632.20.210791.20000053223_mask_pred_float.nii',
 '/media/deeplearning/Elements/AHL2011_CSV_DATA/AHL2011-CSV/poi_is_jun 5_2013-EV.csv',
 '0.009999999999990905']

In [45]:
volume = []
for row in csv_data_3 : 
    volume.append(float(row[-1]))

In [46]:
sorted_vol = sorted(volume)

diff = sorted_vol[-40:]

In [47]:
diff

[65.82000000000005,
 65.98000000000025,
 66.32000000000005,
 66.49999999999997,
 68.36000000000001,
 69.81000000000009,
 70.71999999999991,
 70.87999999999994,
 72.4799999999999,
 73.09,
 74.16999999999996,
 74.47999999999999,
 76.84999999999994,
 77.40000000000009,
 85.42000000000002,
 85.80999999999995,
 91.35000000000002,
 93.45000000000005,
 96.61999999999978,
 96.76999999999998,
 108.4799999999999,
 109.18999999999994,
 114.47,
 115.23000000000008,
 127.44000000000005,
 137.4699999999998,
 141.26999999999992,
 145.15999999999997,
 156.44999999999976,
 160.44000000000028,
 175.00000000000006,
 183.45999999999992,
 190.14000000000033,
 191.7099999999998,
 220.23999999999933,
 242.7199999999999,
 262.78999999999996,
 366.44000000000005,
 465.9500000000003,
 524.7199999999998]

In [48]:
new_liste = []

for numb in diff : 
    for serie in csv_data_3 :
        if str(numb) in serie : 
            new_liste.append(serie)


In [51]:
len(new_liste)

40

In [53]:
new_liste[0]

['53011101371012',
 '1.2.250.1.74.20140306150000.1000040424629',
 '/media/deeplearning/Elements/AHL2011_NIFTI/1.2.250.1.74.20140306150000.1000040424629_nifti_PT.nii',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/inference/1.2.250.1.74.20140306150000.1000040424629_mask_pred_float.nii',
 '/media/deeplearning/Elements/AHL2011_CSV_DATA/AHL2011-CSV/ber_gu_feb 24_2014_SK.csv',
 '65.82000000000005']

In [54]:
#write csv with 40 biggest difference
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/post_processing'

with open(os.path.join(nifti_directory, 'ahl_post_process_40.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID", "NIFTI_PET", "NIFTI_MASK", "CSV", "DIFF"])
    for serie in new_liste: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5]])

In [19]:
#clean GAINED VALIDATED DICOM : PET 0 FAIT DONC ENLEVER LES STUDYs
import shutil

In [14]:
dicom_directory = '/media/deeplearning/Elements/GAINED_Validated_DICOMS'

In [15]:
csv_pet0_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/GAINED_PET0_NIFTI.csv'

with open(csv_pet0_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [17]:
liste = []
for row in csv_data : 
    subliste = []
    subliste.append(row[0])
    subliste.append(row[1])
    liste.append(subliste)


In [20]:
for serie in liste : 
    shutil.rmtree(dicom_directory+'/'+serie[0]+'/'+serie[1])